# ReActAgent

Data Agents are LLM-powered knowledge workers in LlamaIndex that can intelligently perform various tasks over your data, in both a “read” and “write” function. They are capable of the following:

- Perform automated search and retrieval over different types of data - unstructured, semi-structured, and structured.

- Calling any external service API in a structured fashion, and processing the response + storing it for later.

In that sense, agents are a step beyond our query engines in that they can not only “read” from a static source of data, but can dynamically ingest and modify data from a variety of different tools.

ReAct, short for Reasoning and Acting, was first introduced in the paper [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/pdf/2210.03629.pdf).  

ReAct Agent, introduced by LlamaIndex, is an agent-based chat mode built on top of a query engine over your data. ReAct Agent is one of LlamaIndex’s main chat engines. For each chat interaction, the agent enters a reasoning and acting loop:

- First, decide whether to use the query engine tool and which query engine tool to use to come up with appropriate input.
- Query with the query engine tool and observe its output.
- Based on the output, decide whether to repeat the process or give a final response.

We will use ReAct agent to analyze the U.S. government’s financial reports for fiscal years 2020, 2021, 2022, and 2023.

LlamaIndex notebook: https://docs.llamaindex.ai/en/stable/examples/chat_engine/chat_engine_react.html.

## Step 1: Setup the Query Tools

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai_api_key = os.getenv('OPENAI_API_KEY')

Be sure to save the files with file name containing underscore, not dash, otherwise, ReAct agent chat completion won't work.

In [6]:
# !mkdir reports
# !wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2020/executive-summary-2020.pdf -O ./reports/executive_summary_2020.pdf
# !wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2021/executive-summary-2021.pdf -O ./reports/executive_summary_2021.pdf
# !wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2022/executive-summary-2022.pdf -O ./reports/executive_summary_2022.pdf
# !wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2023/executive-summary-2023.pdf -O ./reports/executive_summary_2023.pdf

# !wget https://www.bls.gov/news.release/archives/cpi_07112024.pdf
# !wget https://www.bls.gov/news.release/archives/cpi_06122024.pdf
# !wget https://www.bls.gov/news.release/archives/cpi_05152024.pdf
# !wget https://www.bls.gov/news.release/archives/cpi_04102024.pdf
# !wget https://www.bls.gov/news.release/archives/cpi_03122024.pdf
# !wget https://www.bls.gov/news.release/archives/cpi_02132024.pddf


In [2]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["reports/executive_summary_2023.pdf"]).load_data()

In [3]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [4]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: executive_summary_2023.pdf
file_path: reports/executive_summary_2023.pdf
file_type: application/pdf
file_size: 904959
creation_date: 2024-07-21
last_modified_date: 2024-02-15

1 EXECUTIVE SUMMARY TO THE 2023 FINANCIAL REPORT OF THE U.S. GOVERNMENT


In [5]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)

# query_engine = vector_index.as_query_engine(similarity_top_k=10)

In [6]:
# What is the U.S. government's net operating cost?
# What are the total revenues reported by the U.S. government?
# What are the total assets reported by the U.S. government?
# What is the total national debt reported by the U.S. government?
# response = query_engine.query(
#     "What is the U.S. government's net operating cost?" 
# )
# print(response)

In [6]:
def vector_query(query: str) -> str:
    query_engine = vector_index.as_query_engine(similarity_top_k=10)
    response = query_engine.query(query)
    return response.response

In [7]:
vector_query("Could you summarize the U.S. government's financial report? ")

"The U.S. government's financial report for FY 2023 highlights key financial indicators such as the budget deficit increasing to $1.7 trillion, net operating cost decreasing to $3.4 trillion, and the debt-to-GDP ratio reaching approximately 97 percent by the end of FY 2023. The report discusses the government's financial position, assets, liabilities, and the unsustainable fiscal path it is currently on. It also addresses the need for policy reforms to ensure sustainability, the impact of delaying fiscal policy reform, and the challenges posed by the continuous rise of the debt-to-GDP ratio. Additionally, the report touches on the government's response to climate change and the importance of understanding and addressing the fiscal challenges faced by future generations."

In [8]:
vector_query("What is the U.S. government's net operating cost?")

"The U.S. government's net operating cost is $3.4 trillion."

In [9]:
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

llm = OpenAI(model="gpt-4o-mini", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "What is the U.S. government's net operating cost?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "U.S. government's net operating cost 2023"}
=== Function Output ===
The U.S. government's net operating cost for 2023 decreased by $753.8 billion (18.1 percent) to $3.4 trillion. This decrease was primarily due to significant decreases in non-cash costs, including reductions in losses resulting from changes in assumptions affecting cost and liability estimates for federal employee and veteran benefits programs, as well as reestimates of long-term student loan costs.


## Multi-Document Agent

In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool, ToolMetadata


query_engine_tools = []
for filename in os.listdir("reports"):
    if filename.endswith(".pdf"):
        file_path = os.path.join("reports", filename)

        with open(file_path, "r") as file:
            documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
            print(f"Loaded {len(documents)} documents from {filename}")
            print(filename[:-4]) # print name without extension

            index = VectorStoreIndex.from_documents(documents)
            splitter = SentenceSplitter(chunk_size=512)
            nodes = splitter.get_nodes_from_documents(documents)
            vector_index = VectorStoreIndex(nodes)
            query_engine = vector_index.as_query_engine(similarity_top_k=10)
            
            query_engine_tool = QueryEngineTool(
                query_engine=query_engine,
                metadata=ToolMetadata(
                    name=f"{filename[:-4]}",  # Construct name without extension
                    description=(
                        f"Provides information about the U.S. government financial report {filename[:-4]}"
                    ),
                ),
            )
            query_engine_tools.append(query_engine_tool)

Loaded 10 documents from executive_summary_2022.pdf
executive_summary_2022
Loaded 10 documents from executive_summary_2023.pdf
executive_summary_2023
Loaded 11 documents from executive_summary_2021.pdf
executive_summary_2021
Loaded 11 documents from executive_summary_2020.pdf
executive_summary_2020


### ReAct Agent

In [15]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4")

react_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

In [16]:
response = react_agent.chat("What is the U.S. government's net operating cost from 2020 to 2023?")
print(response)

Thought: The user is asking for the U.S. government's net operating cost from 2020 to 2023. I need to use the executive_summary tools for the years 2020 to 2023 to find this information.
Action: executive_summary_2020
Action Input: {'input': 'net operating cost'}
Observation: The net operating cost increased by $2.4 trillion during FY 2020 to $3.8 trillion.
Thought: I have the net operating cost for 2020. Now I need to find the net operating cost for 2021.
Action: executive_summary_2021
Action Input: {'input': 'net operating cost'}
Observation: The net operating cost decreased by $746.5 billion (19.4 percent) during FY 2021 to $3.1 trillion.
Thought: I have the net operating cost for 2021. Now I need to find the net operating cost for 2022.
Action: executive_summary_2022
Action Input: {'input': 'net operating cost'}
Observation: The net operating cost is calculated by subtracting earned program revenues from total gross costs and adjusting for gains or losses from changes in actuarial 

In [17]:
response = react_agent.chat("Compare and contrast the anual net operating cost from 2020 to 2023, then give an analysis")
print(str(response))

Thought: To compare and contrast the annual net operating cost from 2020 to 2023, I need to use the executive_summary tools for each year to get the detailed information. I'll start with the year 2020.
Action: executive_summary_2020
Action Input: {'input': 'net operating cost'}
Observation: $3.8 trillion
Thought: Now that I have the net operating cost for 2020, I need to get the same information for 2021.
Action: executive_summary_2021
Action Input: {'input': 'net operating cost'}
Observation: The net operating cost decreased by $746.5 billion (19.4 percent) during FY 2021 to $3.1 trillion.
Thought: Now that I have the net operating cost for 2021, I need to get the same information for 2022.
Action: executive_summary_2022
Action Input: {'input': 'net operating cost'}
Observation: The net operating cost is calculated by subtracting earned program revenues from total gross costs and adjusting for gains or losses from changes in actuarial assumptions related to federal employee and vetera

In [58]:
llm_instruct = OpenAI(model="gpt-3.5-turbo-instruct")

react_agent = ReActAgent.from_tools(query_engine_tools, llm=llm_instruct, verbose=True)

In [61]:
response = react_agent.chat("What is the U.S. government's net operating cost for the most recent fiscal year?")
print(str(response))

> Running step 1861d60e-7096-4f22-9748-6243b8f1239a. Step input: What is the U.S. government's net operating cost for the most recent fiscal year?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: executive_summary_2023
Action Input: {'input': 'net operating cost'}
Observation: The net operating cost is calculated by subtracting earned program revenues and adjusting for gains or losses from changes in actuarial assumptions used to estimate future federal employee and veteran benefits payments from the total gross costs. This calculation results in the government's "bottom line" net operating cost.
> Running step f847cb2b-ddc0-48ec-8d19-bf7e862820ab. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The U.S. government's net operating cost for the most recent fiscal year is $1.9 trillion.
The U.S. government's net operating cost for the most recent fiscal y

In [62]:
response = react_agent.chat("Compare and contrast the anual net operating cost from 2020 to 2023, then give an analysis")
print(str(response))

> Running step 7116d296-66a5-4de0-af2e-5f5976fc7255. Step input: Compare and contrast the anual net operating cost from 2020 to 2023, then give an analysis
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The U.S. government's net operating cost for the most recent fiscal year is $1.9 trillion. This is a decrease from the previous fiscal year's net operating cost of $3.8 trillion. This decrease can be attributed to various factors, such as changes in government spending and revenue. Overall, the net operating cost has decreased by $1.9 trillion over the course of four years.
The U.S. government's net operating cost for the most recent fiscal year is $1.9 trillion. This is a decrease from the previous fiscal year's net operating cost of $3.8 trillion. This decrease can be attributed to various factors, such as changes in government spending and revenue. Overall, the net operating cost has decreased by $1.9 trillion over the course of fou